In [ ]:
import os
import sys
import numpy as np
from scipy.sparse import csr_matrix
if '..' not in sys.path:
    sys.path.append('..')
from pfcommon import parse_sparse_matrix_file, parse_Amat_vars_file, parse_Jacobian_vars_file

In [ ]:
def print_matrix(M, var_names, outfile=None):
    n_vars = len(var_names)
    n_char = max(map(len, var_names))
    fmt_str = '  {:>' + str(n_char) + 's} '
    fmt_num = '  {:' + str(n_char) + 'g} '
    if outfile is not None:
        out = open(outfile,'w')
    else:
        out = sys.stdout
    out.write(' ' * 5)
    for var_name in var_names:
        out.write(fmt_str.format(var_name))
    out.write('\n')
    for i,row in enumerate(M):
        out.write(f'[{i+1:02d}] ')
        for val in row:
            out.write(fmt_num.format(val))
        out.write('\n')
    if outfile is not None:
        out.close()

In [ ]:
folder = os.path.join('..','..','modal_analysis','SM_with_load','default')
filename = os.path.join(folder, 'VariableToIdx_Jacobian.txt')
vars_idx,state_vars,voltages,currents,signals = parse_Jacobian_vars_file(filename)
filename = os.path.join(folder, 'Jacobian.mtl')
J = parse_sparse_matrix_file(filename, )
filename = os.path.join(folder, 'SM_with_load_AC.npz')
data = np.load(filename, allow_pickle=True)
S = data['S'].item()
PF = data['PF_without_slack'].item()
PF_bus = PF['buses']['Bus1']
PF_load = PF['loads']['LD1']
PF_gen = PF['SMs']['G1']
cosphi = PF_gen['cosphi']
ϕ = np.arccos(cosphi)
print('cos(ϕ) = {:7.3f}'.format(cosphi))
print('     ϕ = {:7.3f} deg'.format(np.rad2deg(ϕ)))

In [ ]:
flatten = lambda D: [k+'.'+el for k,subl in D.items() for el in subl]
var_names = flatten(state_vars) + flatten(voltages) + flatten(currents)
print(var_names)

#### Base parameters
These values are used by PowerFactory for display purposes

In [ ]:
F      = 50.  # [Hz] default frequency
S_base = 1e6  # [VA] base apparent power
V_base = PF_bus['Vl']/PF_bus['u']*1e3  # [V] base voltage (line-to-line)
I_base = S_base/V_base              # [A] base current
Z_base = V_base**2/S_base           # [Ω] base impedance
Y_base = 1/Z_base                   # [S] base admittance
print('====== System ======')
print('S_base = {:7.3f} MVA'.format(S_base*1e-6))
print('V_base = {:7.3f} kV'.format(V_base*1e-3))
print('I_base = {:7.3f} kA'.format(I_base*1e-3))
print('Z_base = {:7.3f} Ω'.format(Z_base))
print('Y_base = {:7.3f} S'.format(Y_base))

#### Generator parameters
Base parameters of the single generator in the network

In [ ]:
S_base_gen = S['G1']*1e6                      # [VA]
V_base_gen = PF_gen['Vl']*1e3 / PF_gen['u']   # [V]
I_base_gen = S_base_gen/V_base_gen            # [A]
Z_base_gen = V_base_gen**2 / S_base_gen       # [Ω]
Y_base_gen = 1 / Z_base_gen                   # [S]
rstr,xstr = 0.2, 0.4                          # [pu] stator parameters
R_gen,X_gen = rstr*Z_base_gen,xstr*Z_base_gen # [Ω]
Z_gen = R_gen + 1j*X_gen                      # [Ω]
Z_gen_pu = Z_gen/Z_base                       # [pu]
R_gen_pu,X_gen_pu = Z_gen_pu.real, Z_gen_pu.imag
print('===== Generator ====')
print('S_base = {:7.3f} MVA'.format(S_base_gen*1e-6))
print('V_base = {:7.3f} kV'.format(V_base_gen*1e-3))
print('I_base = {:7.3f} kA'.format(I_base_gen*1e-3))
print('Z_base = {:7.3f} Ω'.format(Z_base_gen))
print('Y_base = {:7.3f} S'.format(Y_base_gen))

#### Load parameters
The p.u. values of the load impedance are referred to the system base.

In [ ]:
G = PF_load['P']*1e6 / (PF_bus['Vl']*1e3)**2  # [S]
B = -PF_load['Q']*1e6 / (PF_bus['Vl']*1e3)**2 # [S]
Y_load = G + 1j*B # [S]
Z_load = 1/Y_load # [Ω]
G_pu,B_pu = G/Y_base, B/Y_base # [pu]
print('========== Load =========')
print('G = {:7.3f} S, {:7.3f} pu'.format(G, G_pu))
print('B = {:7.3f} S, {:7.3f} pu'.format(B, B_pu))

## Power flow results
#### Generator

In [ ]:
print('     P = {:7.3f} MW'.format(PF_gen['P']))
print('     Q = {:7.3f} Mvar'.format(PF_gen['Q']))
print('     u = {:7.3f} pu'.format(PF_gen['u']))
print('    ur = {:7.3f} pu'.format(PF_gen['ur']))
print('    ui = {:7.3f} pu'.format(PF_gen['ui']))
print(' V_l2l = {:7.3f} kV'.format(PF_gen['Vl']))
print(' V_l2g = {:7.3f} kV'.format(PF_gen['V']))
print('   V_ϕ = {:7.3f} deg'.format(PF_gen['phiu']))
print('     I = {:7.3f} kA'.format(PF_gen['I']))
print('     i = {:7.3f} pu'.format(PF_gen['i']))
print('    ir = {:7.3f} pu'.format(PF_gen['ir']))
print('    ii = {:7.3f} pu'.format(PF_gen['ii']))

#### Load

In [ ]:
print('     P = {:7.3f} MW'.format(PF_load['P']))
print('     Q = {:7.3f} Mvar'.format(PF_load['Q']))
print('     u = {:7.3f} pu'.format(PF_load['u']))
print('    ur = {:7.3f} pu'.format(PF_load['ur']))
print('    ui = {:7.3f} pu'.format(PF_load['ui']))
print(' V_l2l = {:7.3f} kV'.format(PF_load['Vl']))
print(' V_l2g = {:7.3f} kV'.format(PF_load['V']))
print('   V_ϕ = {:7.3f} deg'.format(PF_load['phiu']))
print('     I = {:7.3f} kA'.format(PF_load['I']))
print('     i = {:7.3f} pu'.format(PF_load['i']))
print('    ir = {:7.3f} pu'.format(PF_load['ir']))
print('    ii = {:7.3f} pu'.format(PF_load['ii']))

#### Bus

In [ ]:
print('     u = {:7.3f} pu'.format(PF_bus['u']))
print('    ur = {:7.3f} pu'.format(PF_bus['ur']))
print('    ui = {:7.3f} pu'.format(PF_bus['ui']))
print(' V_l2l = {:7.3f} kV'.format(PF_bus['Vl']))
print(' V_l2g = {:7.3f} kV'.format(PF_bus['V']))
print('     ϕ = {:7.3f} deg'.format(PF_bus['phi']))

In [ ]:
stop = len(var_names)
print_matrix(J[:stop,:stop], var_names, 'J.out' if stop > 6 else None)

In [ ]:
J_sub_1 = np.array([
    [0., 0., G_pu, -B_pu, -1., 0.],
    [0., 0., B_pu,  G_pu, 0., -1.]
])

V = (Z_gen/Z_base_gen) * (PF_gen['ir'] + 1j*PF_gen['ii'])
e = (PF_bus['ur'] + 1j*PF_bus['ui']) + V
E0,ϕg = np.abs(e), np.angle(e)
J_sub_2 = np.array([
    [0.,  E0*np.sin(ϕg), 1., 0., R_gen_pu, -X_gen_pu],
    [0., -E0*np.cos(ϕg), 0., 1., X_gen_pu,  R_gen_pu]
])

In [ ]:
J_guess = np.zeros((stop,stop))
J_guess[2:4,:] = J_sub_1
J_guess[4:6,:] = J_sub_2
print_matrix(J_guess, var_names, 'J_guess.out' if stop > 6 else None)